Estimated number of reported drink drive accidents and casualties in Great Britain: 1979 - 2016

In [20]:
from gssutils import *

scraper = Scraper('https://www.gov.uk/government/statistical-data-sets/ras51-reported-drinking-and-driving')
scraper

## Reported drinking and driving (RAS51)

Data about the reported drink-drive accidents and casualties, produced by Department for Transport.

### Distributions

1. Reported drink and drive (Excel data tables) ([application/zip](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/827488/drink-drive-ras51.zip))
1. Reported drink drive accidents and casualties in Great Britain since 1979 ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/827490/ras51001.ods))
1. Reported drink drive accidents and casualties, by month ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/827498/ras51011.ods))
1. Reported drink drive accidents and casualties, by country and English region ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/827501/ras51019.ods))
1. Reported drink drive accidents and casualties by gender of driver and rider ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/827502/ras51022.ods))
1. Estimated number of reported road accidents involving a car drink driver, by driver age, accidents per licence holder and per mile driven ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/827497/ras51010.ods))
1. Reported drink drive accidents, by time of day ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/827499/ras51012.ods))
1. Reported drink drive accidents by pedestrian and vehicle involvement ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/827500/ras51013.ods))
1. Estimated number of drink drive casualties in reported accidents by casualty type, gender and age ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/827492/ras51005.ods))
1. KSI casualties in reported accidents involving young drivers and riders (17-24 years old) over the legal alcohol limit ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/827495/ras51008.ods))
1. Driver and rider fatalities in reported accidents: over the legal blood alcohol limit ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/827493/ras51006.ods))
1. Driver and rider fatalities in reported accidents, by BAC category, gender and age ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/827494/ras51007.ods))
1. Blood alcohol levels of reported fatalities aged 16 and over ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/827496/ras51009.ods))
1. Breath tests and breath test failures by drivers and riders involved in reported accidents by vehicle type ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/834413/ras51002.ods))
1. Breath tests and breath test failures in reported accidents by day of week and time of day ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/834414/ras51003.ods))
1. Breath tests and breath test failures in reported accidents by road user type and age ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/834415/ras51004.ods))
1. Drivers and riders involved in reported road accidents: breath tests and failures ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/834416/ras51014.ods))
1. Car drivers involved in reported road accidents: breath tests and failures ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/834418/ras51015.ods))
1. Car drivers in reported road accidents, breath tests and failures by country and English region ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/834423/ras51020.ods))
1. Reported motorcyclist breath tests and failure rates by age ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/834425/ras51021.ods))
1. Reported roadside screening breath tests and breath test failures: England and Wales ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/834419/ras51016.ods))
1. Screening test results, by reason for test ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/834420/ras51017.ods))
1. Results of screening breath tests following a road traffic collision ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/834422/ras51018.ods))
1. Self-reported drink driving, percentage of all drivers ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/834426/ras51101.ods))
1. Self-reported drink driving, percentage of drivers by age and gender ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/834427/ras51102.ods))
1. Self-reported drug driving, percentage of all drivers ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/834428/ras51103.ods))
1. Self-reported drug driving, percentage of drivers by age and gender ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/834430/ras51104.ods))
1. Self-reported mobile phone use as a proportion of all drivers ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/834411/ras51105.ods))
1. Self-reported mobile phone use by age and sex of drivers ([ODF Spreadsheet](https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/834412/ras51106.ods))


In [21]:
try:
    df = scraper.distribution(
        title='Reported drink drive accidents and casualties in Great Britain since 1979'

    ).as_pandas(sheet_name='RAS51001', start_row = 6,
            row_limit = 39, start_column = 0, column_limit = 12)
    df
except Exception as e:
         print(e.message, e.args)

In [22]:
table = pd.DataFrame()

In [23]:
observations = df.iloc[:,[0,1,2,3,4]]
observations = observations.rename(columns=observations.iloc[0]).drop(observations.index[0])
observations.columns.values[0] = 'Period'
observations.columns.values[1] = 'fatal'
observations.columns.values[2] = 'serious'
observations.columns.values[3] = 'slight'
observations.columns.values[4] = 'total'
Final_table = pd.melt(observations,
                       ['Period'], var_name="Severity",
                       value_name="Value")
Final_table['Unit'] = 'accidents'
Final_table['Measure Type'] = 'Count of accidents'
table = pd.concat([table,Final_table])

In [24]:
observations1 = df.iloc[:,[0,7,9,10,11]]
observations1 = observations1.rename(columns=observations1.iloc[0]).drop(observations1.index[0])
observations1.columns.values[0] = 'Period'
observations1.columns.values[1] = 'killed'
observations1.columns.values[2] = 'seriously-injured'
observations1.columns.values[3] = 'slightly-injured'
observations1.columns.values[4] = 'total'
Final_table = pd.melt(observations1,
                       ['Period'], var_name="Severity",
                       value_name="Value")
Final_table['Unit'] = 'casualties'
Final_table['Measure Type'] = 'Count of casualties'
table = pd.concat([table,Final_table])

Now deal with the confidence intervals?

In [25]:
observations3 = df.iloc[:,[0,6,8]]
observations3 = observations3.rename(columns=observations3.iloc[0]).drop(observations3.index[0])
observations3.columns.values[0] = 'Period'
observations3.columns.values[1] = 'CI Lower'
observations3.columns.values[2] = 'CI Upper'
observations3['Severity'] = 'killed'
table = pd.merge(table, observations3, how = 'left', left_on = ['Period','Severity'],
                      right_on = ['Period', 'Severity'])

In [26]:
table = table[ ['Period','Severity', 'Measure Type','Value', 'CI Lower', 'CI Upper', 'Unit']]

In [27]:
table['Period'] = 'year/' + table['Period'].map(str)

In [28]:
import numpy as np
table['CI Lower'] = table['CI Lower'].map(lambda x:
                            '' if x == ':' else x)
table['CI Upper'] = table['CI Upper'].map(lambda x:
                            '' if x == ':' else x)

In [29]:
from pathlib import Path

out = Path('out')
out.mkdir(exist_ok=True, parents=True)
table.drop_duplicates().to_csv(out / ('observations.csv'), index = False)

In [30]:
scraper.dataset.family = 'health'
scraper.dataset.theme = THEME['health-social-care']
with open(out / 'dataset.trig', 'wb') as metadata:
    metadata.write(scraper.generate_trig())

In [31]:
table

,Period,Severity,Measure Type,Value,CI Lower,CI Upper,Unit
0,year/1979,fatal,Count of accidents,1380,NaN,NaN,accidents
1,year/1980,fatal,Count of accidents,1280,NaN,NaN,accidents
2,year/1981,fatal,Count of accidents,1200,NaN,NaN,accidents
3,year/1982,fatal,Count of accidents,1300,NaN,NaN,accidents
4,year/1983,fatal,Count of accidents,950,NaN,NaN,accidents
5,year/1984,fatal,Count of accidents,1000,NaN,NaN,accidents
6,year/1985,fatal,Count of accidents,900,NaN,NaN,accidents
7,year/1986,fatal,Count of accidents,850,NaN,NaN,accidents
8,year/1987,fatal,Count of accidents,780,NaN,NaN,accidents
9,year/1988,fatal,Count of accidents,680,NaN,NaN,accidents


HTTPError: HTTP Error 404: Not Found